In [ ]:
import deepconfig

In [ ]:
from keras.models import Sequential, Graph
from keras.layers.core import Dense, Activation, Merge, Dropout
from keras.optimizers import SGD
import h5py 
import os
import numpy as np

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [1]:
from trackerModel import trackerModel

In [2]:
tracker = trackerModel( nlayers = 4, radius_layer=10, lenght_layer=20., cell_size=1., min_n_tracks=50, max_n_tracks=500, target_3par=True)
val_tracker = trackerModel( nlayers = 4, radius_layer=10, lenght_layer=20., cell_size=1., min_n_tracks=50, max_n_tracks=500, target_3par=True)

shape of layer input 0 (62, 20, 1)
shape of layer target 0 (62, 20, 3)
shape of layer input 1 (125, 20, 1)
shape of layer target 1 (125, 20, 3)
shape of layer input 2 (188, 20, 1)
shape of layer target 2 (188, 20, 3)
shape of layer input 3 (251, 20, 1)
shape of layer target 3 (251, 20, 3)
shape of layer input 0 (62, 20, 1)
shape of layer target 0 (62, 20, 3)
shape of layer input 1 (125, 20, 1)
shape of layer target 1 (125, 20, 3)
shape of layer input 2 (188, 20, 1)
shape of layer target 2 (188, 20, 3)
shape of layer input 3 (251, 20, 1)
shape of layer target 3 (251, 20, 3)


In [3]:
d,t = tracker.make_event()

In [4]:
d.shape

(12520,)

In [5]:
t.shape

(12520, 3)

In [7]:
t[:10]

array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  3.05079071e-02,   4.22486875e-02,   6.39217591e-01],
       [  6.31297947e-01,   7.64663243e+00,   7.37826649e-01],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  5.90255514e-03,   7.46817668e+00,   9.73730739e-01],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00]])

In [8]:
d[:10]

array([ 0.,  1.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.])

In [9]:
from keras.models import Model
from keras.layers import Input, Convolution2D, MaxPooling2D, Flatten, Merge, Dense, Dropout, merge

Using Theano backend.
Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5005)

Couldn't import dot_parser, loading of dot files will not be possible.


In [10]:
i = Input(shape=(tracker.full_size,), name='pixelbits')
o = i 
for ilayer in range(0):
    o = Dense(internal,activation='sigmoid')(o)
pre_merge = Dense(tracker.full_size,activation='sigmoid',name='premerge')(o)
## making a mask of the output by the input
o = merge([i,pre_merge], mode='mul')

model = Model(input=i, output=o)

model.compile(optimizer='adam', loss='mse',metrics = ['accuracy'])


In [11]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
pixelbits (InputLayer)           (None, 12520)         0                                            
____________________________________________________________________________________________________
premerge (Dense)                 (None, 12520)         156762920   pixelbits[0][0]                  
____________________________________________________________________________________________________
merge_1 (Merge)                  (None, 12520)         0           pixelbits[0][0]                  
                                                                   premerge[0][0]                   
Total params: 156762920
____________________________________________________________________________________________________


In [12]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=100)

In [13]:
train_on = 1000
validate_on = 100
run_for = 10000
history = model.fit_generator(tracker.generator(train_on),
                    samples_per_epoch=train_on,
                    nb_epoch=run_for,
                    validation_data=val_tracker.generator(validate_on),
                    nb_val_samples=validate_on,
                    #callbacks=[early_stopping],
                    verbose=False
                   )

Exception: Error when checking model target: expected merge_1 to have 2 dimensions, but got array with shape (1000, 12520, 3)